In [15]:

import sys

sys.path.append("/nfs/nhome/live/aoomerjee/MSc-Thesis/")


from pathlib import Path
from hct.io.model import load, save
import pandas as pd
import jax.numpy as jp
import jax
import importlib.util
import re

def extract_date_time_from_log(log_file_path):
    """
    Extracts the date and time from the first line of the given log file.
    
    Args:
    - log_file_path (str): Path to the log file.

    Returns:
    - str: Date and time from the first log line, or None if not found.
    """
    with open(log_file_path, 'r') as file:
        first_line = file.readline()
        
        # Extract date and time using regex
        match = re.search(r'I(\d{2})(\d{2}) (\d{2}:\d{2}:\d{2}.\d{6})', first_line)
        if match:
            month, day, time = match.groups()
            date_str = f"{month}/{day} {time}"
            return date_str
    return None

def import_function_from_path(path, function_name):
    spec = importlib.util.spec_from_file_location("__temp_module__", path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return getattr(module, function_name)

hyperparam_sweeps = Path("/nfs/nhome/live/aoomerjee/MSc-Thesis/hct/training/hyperparam_sweeps")

hma_filepath = hyperparam_sweeps / "hma"
hurdles_filepath = hyperparam_sweeps / "hurdles"
gaps_filepath = hyperparam_sweeps / "gaps"
maze_filepath = hyperparam_sweeps / "maze"

filepaths = [hma_filepath, hurdles_filepath, gaps_filepath, maze_filepath]


In [16]:
def generate_data_tables():

    data = []
    
    for i, filepath in enumerate(filepaths):

        model_category = filepath.name

        for j, environment in enumerate(filepath.iterdir()):

            environment_name = environment.name

            if environment.is_dir() and not (environment_name.startswith('__') or environment_name.startswith('.')):
                runs = environment / 'runs'
                hyperparameter_sweep = import_function_from_path(environment / 'run.py', 'hyperparameter_sweep')
                env_params, training_params = hyperparameter_sweep()
                
                for run in runs.iterdir():

                    model_variant = run.name

                    if (run / 'network').exists() and int(model_variant) < len(env_params):

                        date = extract_date_time_from_log(run / 'log.INFO')

                        training_metrics = load(f'{run}/training_metrics')

                        env_params[int(model_variant)].pop('architecture_configs')

                        final_metrics = {'date':date, 'model_category': model_category, 'environment': environment_name, 'model_variant_id': model_variant}
                        final_metrics.update(env_params[int(model_variant)])
                        if 'low_level_modelpath' in final_metrics.keys() and environment_name != 'mid_level':
                            low_level_model_name = int(Path(final_metrics['low_level_modelpath']).name)
                            low_level_env_params, _ = import_function_from_path(Path(final_metrics['low_level_modelpath']).parent.parent / 'run.py', 'hyperparameter_sweep')()
                            final_metrics['low_level_modelpath'] = low_level_model_name
                            final_metrics['position_goals'] = low_level_env_params[low_level_model_name]['position_goals']
                            final_metrics['velocity_goals'] = low_level_env_params[low_level_model_name]['velocity_goals']
                            final_metrics['goal_dist'] = low_level_env_params[low_level_model_name]['goal_dist']
                            final_metrics['distance_reward'] = low_level_env_params[low_level_model_name]['distance_reward']
                            final_metrics['goal_importance'] = low_level_env_params[low_level_model_name]['goal_importance']

                        final_metrics.update(training_metrics[list(training_metrics.keys())[-1]])
                        final_metrics = {key: f"{value['mean']:.4f} ± {value['std']:.4f}" if isinstance(value, dict) else value for key, value in final_metrics.items()}

                        data.append(final_metrics)

    data = pd.DataFrame(data)
    data.sort_values(by=['model_category', 'environment', 'model_variant_id'], inplace=True, ascending=True)

    savepath = f"experimental_results.csv"
    data.to_csv(savepath, index=False)

    return data
data = generate_data_tables()

In [4]:
benchmark_task_data = data[(data.model_category == 'gaps') | (data.model_category == 'hurdles') | (data.model_category == 'maze')]

hurdles_data = benchmark_task_data[(data.model_category == 'hurdles')]
gaps_data = benchmark_task_data[(data.model_category == 'gaps')]
maze_data = benchmark_task_data[(data.model_category == 'maze')]

mid_level_data = data[(data.model_category == 'mid_level')]



/tmp/ipykernel_1182759/3953053848.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hurdles_data = benchmark_task_data[(data.model_category == 'hurdles')]
/tmp/ipykernel_1182759/3953053848.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gaps_data = benchmark_task_data[(data.model_category == 'gaps')]
/tmp/ipykernel_1182759/3953053848.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  maze_data = benchmark_task_data[(data.model_category == 'maze')]


In [18]:
maze_data[maze_data.reward_type == 'sparse'].sort_values(by='eval/episode_distance_to_end').head(50)

,date,model_category,environment,model_variant_id,position_goals,velocity_goals,goal_dist,distance_reward,goal_importance,eval/walltime,...,reward_type,task_information,reward_movement,action_repeat,eval/episode_obstacles_complete,eval/episode_task_complete,mid_level_modelpath,eval/episode_distance_to_end,eval/episode_maze_column,eval/episode_maze_row
116,08/21 14:58:42.082834,maze,flat,2,NaN,NaN,NaN,NaN,NaN,434.966751,...,sparse,True,NaN,NaN,NaN,1.0000 ± 0.0000,NaN,0.0000 ± 0.0000,1.0000 ± 0.0000,1.0000 ± 0.0000
131,08/22 17:05:32.090215,maze,hma2,8,True,NaN,relative,relative,None,493.653976,...,sparse,True,velocity,1.0,NaN,0.9844 ± 0.1240,NaN,0.1484 ± 1.4312,1.0312 ± 0.2778,1.0234 ± 0.1962
164,08/22 22:14:46.135955,maze,hma2,40,True,NaN,relative,relative,discrete,700.728705,...,sparse,True,velocity,1.0,NaN,0.9609 ± 0.1937,NaN,0.4141 ± 2.3931,1.0625 ± 0.4463,1.0703 ± 0.4180
136,08/22 18:05:40.355314,maze,hma2,13,True,NaN,relative,relative,None,464.944272,...,sparse,False,velocity,5.0,NaN,0.0000 ± 0.0000,NaN,11.4453 ± 1.6993,3.6641 ± 0.5043,4.5000 ± 0.9100
137,08/22 17:35:36.749408,maze,hma2,12,True,NaN,relative,relative,None,495.031034,...,sparse,False,velocity,1.0,NaN,0.0000 ± 0.0000,NaN,11.4922 ± 2.1902,3.5156 ± 0.5152,4.3984 ± 1.1946
150,08/22 19:59:29.187330,maze,hma2,26,True,NaN,relative,relative,continuous,559.616409,...,sparse,True,velocity,10.0,NaN,0.0000 ± 0.0000,NaN,11.6250 ± 2.0842,3.7422 ± 0.5191,4.4141 ± 0.9963
169,08/22 23:08:15.217974,maze,hma2,45,True,NaN,relative,relative,discrete,549.617429,...,sparse,False,velocity,5.0,NaN,0.0000 ± 0.0000,NaN,11.8203 ± 1.7960,3.8906 ± 0.3999,4.3984 ± 0.8780
170,08/22 23:19:15.763782,maze,hma2,46,True,NaN,relative,relative,discrete,644.459745,...,sparse,False,velocity,10.0,NaN,0.0000 ± 0.0000,NaN,11.8438 ± 1.7475,3.8281 ± 0.4350,4.2812 ± 0.9432
151,08/22 20:19:07.405837,maze,hma2,27,True,NaN,relative,relative,continuous,546.422463,...,sparse,True,velocity,20.0,NaN,0.0000 ± 0.0000,NaN,11.8828 ± 1.9749,3.7969 ± 0.4737,4.2891 ± 0.9450
139,08/22 18:17:11.081682,maze,hma2,14,True,NaN,relative,relative,None,458.873421,...,sparse,False,velocity,10.0,NaN,0.0000 ± 0.0000,NaN,11.8906 ± 2.1587,3.6562 ± 0.5511,4.1719 ± 1.1396


In [22]:
hurdles_data[hurdles_data.reward_type == 'sparse'].sort_values(by='eval/episode_obstacles_complete', ascending=False).head(50)

,date,model_category,environment,model_variant_id,position_goals,velocity_goals,goal_dist,distance_reward,goal_importance,eval/walltime,...,reward_type,task_information,reward_movement,action_repeat,eval/episode_obstacles_complete,eval/episode_task_complete,mid_level_modelpath,eval/episode_distance_to_end,eval/episode_maze_column,eval/episode_maze_row
54,08/22 00:38:36.326975,hurdles,flat,2,NaN,NaN,NaN,NaN,NaN,524.441337,...,sparse,True,NaN,NaN,6.8203 ± 2.6588,0.1328 ± 0.3394,NaN,NaN,NaN,NaN
44,08/22 07:23:23.796067,hurdles,hma2,41,True,NaN,relative,relative,discrete,419.216336,...,sparse,True,velocity,5.0,1.4375 ± 0.6092,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
28,08/22 05:46:45.325486,hurdles,hma2,25,True,NaN,relative,relative,continuous,424.142921,...,sparse,True,velocity,5.0,1.0703 ± 0.3350,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
45,08/22 07:24:05.762242,hurdles,hma2,42,True,NaN,relative,relative,discrete,522.280771,...,sparse,True,velocity,10.0,0.9531 ± 0.4654,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
49,08/22 07:47:27.965115,hurdles,hma2,46,True,NaN,relative,relative,discrete,412.849939,...,sparse,False,velocity,10.0,0.9453 ± 0.3379,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
16,08/23 01:36:05.618926,hurdles,hma2,13,True,NaN,relative,relative,None,341.169833,...,sparse,False,velocity,5.0,0.8828 ± 0.3216,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
29,08/22 05:49:16.508623,hurdles,hma2,26,True,NaN,relative,relative,continuous,565.504104,...,sparse,True,velocity,10.0,0.8281 ± 0.4166,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
43,08/22 07:22:00.931105,hurdles,hma2,40,True,NaN,relative,relative,discrete,618.771665,...,sparse,True,velocity,1.0,0.8125 ± 0.3903,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
6,08/23 01:10:46.166334,hurdles,hma2,9,True,NaN,relative,relative,None,469.055557,...,sparse,True,velocity,5.0,0.6797 ± 0.5294,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
32,08/22 06:10:56.580337,hurdles,hma2,29,True,NaN,relative,relative,continuous,416.278531,...,sparse,False,velocity,5.0,0.5078 ± 0.5153,0.0000 ± 0.0000,NaN,NaN,NaN,NaN


In [24]:
gaps_data.sort_values(by='eval/episode_obstacles_complete', ascending=False).head(50)

,date,model_category,environment,model_variant_id,position_goals,velocity_goals,goal_dist,distance_reward,goal_importance,eval/walltime,...,reward_type,task_information,reward_movement,action_repeat,eval/episode_obstacles_complete,eval/episode_task_complete,mid_level_modelpath,eval/episode_distance_to_end,eval/episode_maze_column,eval/episode_maze_row
56,08/22 00:38:54.054228,gaps,flat,1,NaN,NaN,NaN,NaN,NaN,751.739275,...,dense,False,NaN,NaN,8.9141 ± 3.7313,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
75,08/22 10:09:13.264787,gaps,hma2,16,True,NaN,relative,relative,continuous,725.212607,...,dense,True,velocity,1.0,7.0469 ± 2.8446,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
58,08/22 00:38:54.195640,gaps,flat,0,NaN,NaN,NaN,NaN,NaN,756.044894,...,dense,True,NaN,NaN,6.2109 ± 1.9310,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
63,08/22 07:51:09.142734,gaps,hma2,0,True,NaN,relative,relative,None,582.100618,...,dense,True,velocity,1.0,5.1172 ± 2.0977,0.0078 ± 0.0880,NaN,NaN,NaN,NaN
91,08/22 12:37:17.063112,gaps,hma2,32,True,NaN,relative,relative,discrete,802.154035,...,dense,True,velocity,1.0,3.9766 ± 1.7742,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
61,08/22 08:11:37.542715,gaps,hma2,4,True,NaN,relative,relative,None,797.034594,...,dense,False,velocity,1.0,3.7969 ± 2.4792,0.0156 ± 0.1240,NaN,NaN,NaN,NaN
95,08/22 13:22:41.582896,gaps,hma2,36,True,NaN,relative,relative,discrete,532.662251,...,dense,False,velocity,1.0,3.6406 ± 1.5946,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
79,08/22 10:28:13.011176,gaps,hma2,20,True,NaN,relative,relative,continuous,530.737640,...,dense,False,velocity,1.0,3.5234 ± 1.5308,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
96,08/22 13:25:24.282809,gaps,hma2,37,True,NaN,relative,relative,discrete,626.669966,...,dense,False,velocity,5.0,1.1094 ± 0.8860,0.0000 ± 0.0000,NaN,NaN,NaN,NaN
92,08/22 13:00:32.891408,gaps,hma2,33,True,NaN,relative,relative,discrete,690.186450,...,dense,True,velocity,5.0,1.0469 ± 0.9258,0.0000 ± 0.0000,NaN,NaN,NaN,NaN


In [29]:
mid_level_data.sort_values(by='eval/episode_reward').head(50)

,model_category,environment,model_variant_id,position_goals,velocity_goals,goal_dist,distance_reward,goal_importance,eval/walltime,training/sps,...,low_level_modelpath,reward_type,reward_movement,action_repeat,eval/episode_distance_to_end,eval/episode_obstacles_complete,eval/episode_task_complete,mid_level_modelpath,eval/episode_maze_column,eval/episode_maze_row
